# 1.2.3: Bikeshare (Stepping Forward in Time)

<br>



---



*Modeling and Simulation in Python*


Copyright 2021 Allen Downey, (License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/))

Revised, Mike Augspurger (2021-present)

<br>

---





## Incorporating Chance into the Simulation

We have a way to keep track of the state of the system (*state variables*), and a rule for how things will change when a bike is moved (*change function*).   But how do we know if a bike is actually moved in a given time step?  This notebook incorporates random chance into our simulation.

### Simulating chance with `if` and comparison operators

As our investigation showed the the arrival of a customer is somewhat random.  Because of this we'll create a *stochastic* rule: that is, we will use a random number generator to determine when customers arrive at each station.

<br>

The Numpy library provides a function `random()` that produces a number between 0 and 1.0 whenever it is called.  Take her for a test drive:

In [ ]:
import numpy.random as npr
npr.random()


0.7876942939946784

How can we use this to determine if a bike leaves a rack during a time step?  We don't really want a random number: we want a "True" or a "False", which is called a *Boolean* value in computing.  We can turn our random number into a *Boolean* by using a *comparison operator*, like `<`.

<br>

<center>
<img src = https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Images/1_2/Random.PNG width = 300>
</center>

<br>

What's the chance that a random number between 0 and 1 is less than 0.5? 50%, right?  Run this cell 10 times, and count how often it is `True`:

In [ ]:
npr.random() < 0.5

True


Now we can use boolean values to control the behavior of the program by using an *if statement*:

In [ ]:
if npr.random() < 0.5:
    print('heads')

heads


If the result is `True`, the program displays the string
`'heads'`. Otherwise it does nothing.

<br>

The syntax for `if` statements is similar to the syntax for
function definitions: the first line has to end with a colon, and the
lines inside the `if` statement have to be indented.
 We can also add an *else clause* to indicate what should
happen if the result is `False`:

In [ ]:
if npr.random() < 0.5:
    print('heads')
else:
    print('tails')

tails


If you run the previous cell a few times, it should print `heads` about half the time, and `tails` about half the time.



We're using `<` here, but here are the other comparison operators:

| Operation             	| Symbol 	|
|-----------------------	|--------	|
| Less than             	| `<`      	|
| Greater than          	| `>`      	|
| Less than or equal    	| `<=`     	|
| Greater than or equal 	| `>=`     	|
| Equal                 	| `==`     	|
| Not equal             	| `!=`     	|


Notice that `==` (a comparison operator) is not the same as `=`, which assigns values.   Observe the difference:

In [ ]:
# Assigns x the value of 5
x = 5

# Checks to see if x is equal to 6
x == 6

False

If you make a mistake and use `=` in an `if` statement, like this, you'll get a *syntax error*:

```
if x = 5:
    print('yes, x is 5')
```

Python will print an error message and the program won't run.

### Using `if` in our change function

Now we can use `random()` to simulate the arrival of customers who want to
borrow a bike. We decided that the chance that a student will check out a bike from Augie in any given 15 minutes is 50%.  We can *implement* that observation in our simulation like this:

In [ ]:
# Create a series to hold the state variables
import pandas as pd
state = dict(augie=10,moline=2)
bikeshare = pd.Series(state,name='Number of Bikes')

# Define the bike_to_moline function
def bike_to_moline():
    bikeshare.moline += 1
    bikeshare.augie -= 1

# Check to see if anyone picks up a bike at Augie
if npr.random() < 0.5:
    bike_to_moline()

Notice that I've included the Series code and function definition from the previous chapter, since those are not loaded in this notebook.

<br>

Similarly, we learned that the chance of a student checking out a bike in Moline in any 15 minute window is 40%:

In [ ]:
def bike_to_augie():
    bikeshare.moline -= 1
    bikeshare.augie += 1

if npr.random() < 0.4:
    bike_to_augie()

We can combine these snippets into a function that simulates a single time step:

In [ ]:
# A more interesting change function
def change_func():
    if npr.random() < 0.5:
        bike_to_moline()

    if npr.random() < 0.4:
        bike_to_augie()

Then we can simulate a time step like this:

In [ ]:
import pandas as pd

# Notice that this creates the state at the same time it
# creates the bikeshare Series
bikeshare = pd.Series(dict(augie=10,moline=2),name='Number of Bikes')
change_func()

bikeshare

augie     10
moline     2
Name: Number of Bikes, dtype: int64

Depending on the results, this function might move a bike to augie, or to moline, or neither, or both.  Run change_func multiple times and note how the state of the system changes.

<br>

`change_func` is our first fully-formed change function!  We can run this function $n$ number of times, and it will simulate the changes in the state of the system over a period of $n*15$ minutes.

### Using parameters to add flexibilty to a change function

The previous version of `change_func` is fine if the arrival probabilities
never change, but in reality they may vary over time, especially as our simulation gets more complex.  So instead of putting the constant values 0.5 and 0.4 in `step`, we can replace them with *arguments*.

<br>

In a function, arguments are variables whose values are set only when a function is called. (Note: these are sometimes called *parameters*, and there is a difference between *parameters* and *arguments*, but we'll ignore that to avoid confusion with the modeling definition of *parameters*).

<br>

Here's a version of `change_func` that takes two arguments, `ptm` and `pta`:

In [ ]:
# Call the arguments something that helps you remember what they do
# btm --> short for "percentage to moline"
def change_func(ptm, pta):
    if npr.random() < ptm:
        print("Bike to Moline")
        bike_to_moline()

    if npr.random() < pta:
        print("Bike to Augie")
        bike_to_augie()

The values of `ptm` and `pta` are not set inside this function; instead,
they are provided when the function is called, like this:

In [ ]:
# Run this multiple times to see how it works
change_func(0.5, 0.4)

Bike to Augie


The arguments, `0.5` and `0.4` in this example, get
assigned to the variables, `ptm` and `pta`, in order. So inside the function, `ptm = 0.5` and `pta = 0.4`.  The advantage of using arguments is that you can call the same function many times, providing different values each time.

<br>

Adding arguments to a function is called *generalization*, because it makes the function more general; without arguments, the function always does the same thing; with them, it can do a range of things.

### Incremental Development

When you start writing programs that are more than a few lines, you
might find yourself spending more time debugging. The more code you write before you start debugging, the harder it is to find the problem.

<br>

*Incremental development* is a way of programming that tries to
minimize the pain of debugging. You might not have noticed, but we've been doing incremental development already.  

<br>

The fundamental steps are:

1.  Start with a working program. If you have an example from a
book, or a program you wrote that is similar to what you are working
on, start with that. Otherwise, start with something you *know* is
correct, like a simple statement of your parameters.  In the section above, for example, we started with a working version of `change_func()`.

2.  Make one small, testable change at a time. A "testable" change is one that displays something or has some other effect you can check. Above, we added arguments to our function.

3.  Run the program and see if the change worked. In the case above, we compared the results of our new function to the results of the old one.  

4. If the change did worked, go back to Step 2 with another alteration. If it didn't, you have to do some debugging, but if the
change you made was small, it shouldn't take long to find the problem.

In the exercises in this class, if you find yourself writing more than a few lines of code before you start testing, remember to take small steps and constantly check the results of your changes!

---

<br>

## Exercises

<br>

---


---
<br>

🟨 🟨

### Exercise 1

A Parson's Problem is a coding exercise that asks you to unscramble code so that it functions correctly.  In the cell below, the code and documentation for a change function is scrambled.   You don't need to add any code: just reorder the lines and fix the indentations.

<br>

Try to do this without looking at the code above; but if you get stuck, you can look back at the notebook.

In [ ]:
if npr.random() < ptm:
print("Bike to Augie")
# Define a change function for the bikeshare model
bike_to_augie()
# Determine if a bike is picked up in Moline
print("Bike to Moline")
bike_to_moline()
if npr.random() < pta:
# Determine if a bike is picked up at Augie
def change_func_scrambled(ptm, pta):


In [ ]:
# If your function is correctly defined, this
# cell will run correctly without changes
bikeshare = pd.Series(dict(augie=10,moline=2),name='Number of Bikes')
change_func_scrambled()
bikeshare